<a href="https://colab.research.google.com/github/Lanzero225/Python-Notebooks/blob/main/Philippine%20Weather/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:

directories = pd.period_range('2023-11', '2025-12', freq='M').strftime('%Y%m').tolist()
dfs = []
#Sample File Link
#https://github.com/Lanzero225/Python-Notebooks/blob/main/Philippine%20Weather/Data/202311_CombinedData.csv

for i in directories:
    url = (
        "https://raw.githubusercontent.com/"
        "Lanzero225/Python-Notebooks/main/"
        f"Philippine%20Weather/Data/{i}_CombinedData.csv"
    )
    dfs.append(pd.read_csv(url))

weather_dataframe = pd.concat(dfs, ignore_index=True)

In [3]:
weather_dataframe.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 27 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   visibility            1512148 non-null  float64
 1   datetime              1513792 non-null  object 
 2   coord.lon             447165 non-null   float64
 3   coord.lat             447165 non-null   float64
 4   main.temp             1513792 non-null  float64
 5   main.feels_like       1513792 non-null  float64
 6   main.temp_min         1513792 non-null  float64
 7   main.temp_max         1513792 non-null  float64
 8   main.pressure         1513792 non-null  float64
 9   main.humidity         1513792 non-null  float64
 10  main.sea_level        1472164 non-null  float64
 11  main.grnd_level       1472164 non-null  float64
 12  wind.speed            1513792 non-null  float64
 13  wind.deg              1513792 non-null  float64
 14  wind.gust             1295449 non-

## Data Definition

- visibility - Measure of distance at which objects can be discerned (0 to 10 km).
- datetime - Date and time of current tracked weather.
- coord.lon - Longitude coordinate of city.
- coord.lat - Latitude coordinate of city.

The following describe the basic units
- main.temp - Temperature in Celsius.
- main.feels_like - Human perception of temperature in Celsius.
- main.pressure/main.sea_level - Atmospheric pressure on the sea level, hPa.
- main.humidity - Humidity, %.
- main.temp_min - Minimum temperature at the moment.
- main.temp_max - Maximum temperature at the moment.
main.grnd_level Atmospheric pressure on the ground level, hPa
city
city.id City ID. Please note that built-in geocoder functionality has been deprecated. Learn more here
city.name City name. Please note that built-in geocoder functionality has been deprecated. Learn more here
city.coord
city.coord.lonGeo location, longitude
city.coord.lat Geo location, latitude
city.country Country code (GB, JP etc.). Please note that built-in geocoder functionality has been deprecated. Learn more here
timezoneShift in seconds from UTC
city.sun
city.sun.rise Sunrise time
city.sun.set Sunset time
temperature
temperature.value Temperature
temperature.min Minimum temperature at the moment of calculation. This is minimal currently observed temperature (within large megalopolises and urban areas), use this parameter optionally. Please find more info here
temperature.max Maximum temperature at the moment of calculation. This is maximal currently observed temperature (within large megalopolises and urban areas), use this parameter optionally. Please find more info here
temperature.unit Unit of measurements. Possible value is Celsius, Kelvin, Fahrenheit
feels_like
feels_like.value Temperature. This temperature parameter accounts for the human perception of weather
feels_like.unit Unit of measurements. Possible value is Celsius, Kelvin, Fahrenheit. Unit Default: Kelvin
humidity
humidity.value Humidity value
humidity.unit Humidity units, %
pressure
pressure.value Pressure value
pressure.unit Pressure units, hPa
wind
wind.speed
wind.speed.value Wind speed
wind.speed.unit Wind speed units, m/s
wind.speed.name Type of the wind
wind.direction
wind.direction.value Wind direction, degrees (meteorological)
wind.direction.code Code of the wind direction. Possible value is WSW, N, S etc.
wind.direction.name Full name of the wind direction
clouds
clouds.value Cloudiness
clouds.name Name of the cloudiness

precipitation
precipitation.value Precipitation, mm. Please note that only mm as units of measurement are available for this parameter.
precipitation.mode Possible values are 'no", name of weather phenomena as 'rain', 'snow'
weather
weather.number Weather condition id
weather.value Weather condition name
weather.icon Weather icon id
lastupdate
lastupdate.value Last time when data was updated

In [5]:
columns = list(weather_dataframe.columns)
weather_dataframe[['main.pressure', 'main.sea_level']]

,main.pressure,main.sea_level
0,1009.0,1009.0
1,1010.0,NaN
2,1010.0,1010.0
3,1009.0,1009.0
4,1009.0,NaN
...,...,...
1513787,1009.0,1009.0
1513788,1010.0,1010.0
1513789,1008.0,1008.0
1513790,1010.0,1010.0


In [54]:
categorical_weather_labels = ['weather.main','weather.description','weather.id','weather.icon']
time_location_labels = ['datetime','sys.sunrise','sys.sunset','coord.lat','coord.lon','city_name']
weather_signal_labels = [
    'main.temp','main.feels_like','main.temp_min','main.temp_max',
    'main.pressure','main.humidity','main.sea_level','main.grnd_level',
    'wind.speed','wind.deg','wind.gust','clouds.all','visibility','rain.1h'
]
system_data = ['sys.type','sys.id','extraction_date_time', 'weather.icon']

In [55]:
weather_dataframe.drop(system_data, inplace=True, axis=1)

In [56]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   visibility           1512148 non-null  float64
 1   datetime             1513792 non-null  object 
 2   coord.lon            447165 non-null   float64
 3   coord.lat            447165 non-null   float64
 4   main.temp            1513792 non-null  float64
 5   main.feels_like      1513792 non-null  float64
 6   main.temp_min        1513792 non-null  float64
 7   main.temp_max        1513792 non-null  float64
 8   main.pressure        1513792 non-null  float64
 9   main.humidity        1513792 non-null  float64
 10  main.sea_level       1472164 non-null  float64
 11  main.grnd_level      1472164 non-null  float64
 12  wind.speed           1513792 non-null  float64
 13  wind.deg             1513792 non-null  float64
 14  wind.gust            1295449 non-null  float64
 15

In [57]:
time_labels = ['datetime', 'sys.sunrise', 'sys.sunset', 'extraction_date_time']
for col in time_labels:
    if col in weather_dataframe.columns:
        weather_dataframe[col] = pd.to_datetime(weather_dataframe[col], errors='coerce')


In [58]:
weather_dataframe[['coord.lat', 'coord.lon']] = (
    weather_dataframe.groupby('city_name')[['coord.lat', 'coord.lon']]
      .transform(lambda x: x.fillna(x.mean()))
)


In [59]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1512148 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1472164 non-null  flo

In [60]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1512148 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1472164 non-null  flo

In [61]:
cities = list(weather_dataframe['city_name'])
print("There are", len(cities), "cities.")
print("The first five are:", cities[:5])

There are 1513792 cities.
The first five are: ['Alaminos', 'Angeles City', 'Antipolo', 'Bacolod', 'Bacoor']


In [62]:
def interpolate_columns_by_city(df, columns, method='linear'):
    df = df.sort_values(['city_name', 'datetime'])
    for col in columns:
        df[col] = (
            df.groupby('city_name')[col]
              .transform(lambda s: s.interpolate(method=method, limit_direction='both'))
        )
    return df

weather_dataframe = interpolate_columns_by_city(
    weather_dataframe,
    ['main.sea_level', 'main.grnd_level', 'wind.gust', 'visibility']
)

In [63]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1513792 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1513792 non-null  float64 

In [65]:
weather_dataframe[(weather_dataframe['weather.id'].isnull())][['weather.id', 'weather.main', 'weather.description']]

,weather.id,weather.main,weather.description
447165,NaN,Clouds,overcast clouds
447303,NaN,Clouds,broken clouds
447441,NaN,Clouds,broken clouds
447579,NaN,Rain,light rain
447717,NaN,Clouds,broken clouds
...,...,...,...
1513246,NaN,Clouds,broken clouds
1513380,NaN,Clouds,broken clouds
1513517,NaN,Clouds,scattered clouds
1513655,NaN,Clouds,broken clouds


In [66]:
weather_dataframe['weather.description'].unique()

array(['overcast clouds', 'light rain', 'scattered clouds', 'clear sky',
       'few clouds', 'broken clouds', 'moderate rain',
       'heavy intensity rain', 'very heavy rain', 'haze',
       'light intensity shower rain', 'fog', 'mist', 'thunderstorm',
       'thunderstorm with light rain', 'thunderstorm with heavy rain',
       'thunderstorm with rain', 'light thunderstorm',
       'heavy intensity shower rain', 'shower rain', 'smoke',
       'light intensity drizzle', 'extreme rain', 'drizzle'], dtype=object)

In [67]:
weather_code_dict = {
    200: 'thunderstorm with light rain',
    201: 'thunderstorm with rain',
    202: 'thunderstorm with heavy rain',
    210: 'light thunderstorm',
    211: 'thunderstorm',
    500: 'light rain',
    501: 'moderate rain',
    502: 'heavy intensity rain',
    503: 'very heavy rain',
    504: 'extreme rain',
    520: 'light intensity shower rain',
    521: 'shower rain',
    522: 'heavy intensity shower rain',
    531: 'ragged shower rain',
    300: 'light intensity drizzle',
    301: 'drizzle',
    302: 'heavy intensity drizzle',
    310: 'light intensity drizzle rain',

    701: 'mist',
    711: 'smoke',
    721: 'haze',
    741: 'fog',


    800: 'clear sky',
    801: 'few clouds',
    802: 'scattered clouds',
    803: 'broken clouds',
    804: 'overcast clouds'
}

In [68]:
description_to_id_dict = {v: k for k, v in weather_code_dict.items()}
weather_dataframe['weather.id'] = weather_dataframe.apply(
    lambda row:
        description_to_id_dict.get(row['weather.description']) if pd.isna(row['weather.id']) else row['weather.id'],
    axis=1
)


In [70]:
weather_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1513792 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1513792 non-null  float64 

In [74]:
weather_dataframe['rain.1h'].fillna(0, inplace=True)
weather_dataframe.info()

/tmp/ipython-input-972124671.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_dataframe['rain.1h'].fillna(0, inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1513792 entries, 0 to 1513791
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype                    
---  ------               --------------    -----                    
 0   visibility           1513792 non-null  float64                  
 1   datetime             1513792 non-null  datetime64[ns, UTC+08:00]
 2   coord.lon            1513792 non-null  float64                  
 3   coord.lat            1513792 non-null  float64                  
 4   main.temp            1513792 non-null  float64                  
 5   main.feels_like      1513792 non-null  float64                  
 6   main.temp_min        1513792 non-null  float64                  
 7   main.temp_max        1513792 non-null  float64                  
 8   main.pressure        1513792 non-null  float64                  
 9   main.humidity        1513792 non-null  float64                  
 10  main.sea_level       1513792 non-null  float64 